In [ ]:
import nltk
#Stemming vs Lemmatization
a = 'wolves'
#nltk.download('all') -->should be done to have all corpus
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
#lemmatization
wnl = WordNetLemmatizer()
wnl.lemmatize(a)
#Stemming
ps = PorterStemmer()
ps.stem(a)
#Understanding Stopwords and Tf-idf Vectorization
from nltk.corpus import stopwords
stopwords
#nltk.download('stopwords') this should be done if above stopwords 
#are not loaded
stopwords.words("english")
#Give your desired sentence or group of words
a = ["We","are","in","ComputerLab","in","the","University"]
#List Comprehension
#a= [exprsn for var in function]
c=[]
for i in a:
   #print(i.upper())
    c.append(i.upper())
print(c)
d =[i.upper() for i in a]
d
e=[]
for i in range(1,11):
    #print(i**2)
    e.append(i**2)
print(e)
d = [i**2 for i in range(1,11)]
d
a
english_stop = set(stopwords.words('english'))
#print(english_stop)
final_data = [i for i in a if i not in english_stop]
final_data
#Tf-idf Vectorization and Cosine Similarity

### TF-IDF Approach
#A problem with the Bag of Words approach is that highly frequent words start to dominate in the document (e.g. larger score), but may not contain as much “informational content”. Also, it will give more weight to longer documents than shorter documents.

#One approach is to rescale the frequency of words by how often they appear in all documents so that the scores for frequent words like “the” that are also frequent across all documents are penalized. This approach to scoring is called Term Frequency-Inverse Document Frequency, or TF-IDF for short, where:

"""**Term Frequency: is a scoring of the frequency of the word in the current document.**"""

'''
TF = (Number of times term t appears in a document)/(Number of terms in the document)


**Inverse Document Frequency: is a scoring of how rare the word is across documents.**

```
IDF = 1+log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.
```
### Cosine Similarity

Tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus

```
Cosine Similarity (d1, d2) =  Dot product(d1, d2) / ||d1|| * ||d2||
```
where d1,d2 are two non zero vectors.

#Steps involved in Building Chatbot
-->Installing packages (nltk)
-->Getting the raw data (create our own corpus)
-->Preprocessing the text (Tokenization,Removal of stopwords,
Stemming & Lemmatization)
-->Keyword Matching
-->Generate Responses
#import the packages
'''
import nltk
import random
from gtts import gTTS
import speech_recognition as sr
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import playsound
import warnings
warnings.filterwarnings('ignore')
#pip install numpy
#pip install scikit-learn
import numpy
from time import ctime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os.path
#pip install -U scikit-learn
#pip install -U numpy
#Read the raw data from the text file
f = open('chatbot.txt','r',errors="ignore") #default we have 'r' mode
#f.read()
#Convert the entire raw text into lower case
f = f.read() #this gives the entire data from text file in string format
#type(f)
f=f.lower() #to avoid case-sensitive errors
'''
The main issue with text data is that it is all in text format (strings). However, the Machine learning algorithms need some sort of numerical feature vector in order to perform the task. So before we start with any NLP project we need to pre-process it to make it ideal for working. Basic text pre-processing includes:

* Converting the entire text into **uppercase** or **lowercase**, so that the algorithm does not treat the same words in different cases as different

* **Tokenization**: Tokenization is just the term used to describe the process of converting the normal text strings into a list of tokens i.e words that we actually want. Sentence tokenizer can be used to find the list of sentences and Word tokenizer can be used to find the list of words in strings.
#Processing the rawtext -->Tokenization
#WordTokenization and SentenceTokenization'''
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
word_tokens = word_tokenize(f)
#word_tokens
sent_tokens = sent_tokenize(f)
sent_tokens[0]
## Preprocessing

#We shall now define a function called LemTokens which will take as input the tokens and return normalized tokens.
#Now we have tokens we will create a function to apply Lemmatization'
lemmer = WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
import string
#to remove punctuation marks and other special character cases
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
#The below function will accept lemmatized words and removes punctuations
def LemNormalize(text):
    return LemTokens(word_tokenize(text.lower().translate(remove_punct_dict)))
## Keyword matching

#Next, we shall define a function for a greeting by the bot i.e if a user’s input is a greeting, the bot shall return a greeting response.ELIZA uses a simple keyword matching for greetings. We will utilize the same concept here.
#Keyword Matching by giving some basic input responses and getting greetings
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", 
                      "I am glad! You are talking to me"]
#Function to generate random responses from above greetings
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
#greeting("hey hai how are you?")
#To generate a response from our bot for input questions, the concept of document similarity will be used. We define a function response which searches the user’s utterance for one or more known keywords and returns one of several possible responses. If it doesn’t find the input matching any of the keywords, it returns a response:” I am sorry! I don’t understand you”
#Generating Responses 
def response(user_response): #here the argument will be user data
    robo_response='' #this will be chatbot response
    sent_tokens.append(user_response) #sent_tokens ->Sentence Tokenizer
    
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, 
                               stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if(req_tfidf==0): #this will be activated if the given response is
        #not present in the corpus 
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response  
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response
#Copy both listen() and respond() function from VirtualAssistant
def listen():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Start talking") #own statement
        audio = r.listen(source,phrase_time_limit=5)
    data = ""
    #Exception Handling
    try:
        data = r.recognize_google(audio,language='en-US')
        print("You said:"+data)
    except sr.UnknownValueError:
        print("I cannot hear you")
    except sr.RequestError as e:
        print("Request Failed")
    return data 
#To respond back with audio 
def respond(String):
    print(String)
    tts = gTTS(text = String,lang = 'en-US')
    if os.path.exists("speech.mp3"):
        os.remove('speech.mp3')
        tts.save('speech.mp3')
    else:
        tts.save("speech.mp3")
    playsound.playsound("speech.mp3")
#Final chat with our chatbot
flag=True #initialized varaible to make loop going ahead
respond("My name is chitti.I will answer your queries about Chatbots.\
        If you want to exit, say Bye!")
while(flag==True):
    user_response = listen()
    user_response=user_response.lower()   
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            respond("You are welcome..")        
        elif 'how are you' in user_response:
            respond("I am Fine,thanks for asking")
        elif 'movie' in user_response:
            respond("I do not watch many movies.Thanks for the concern")
        elif 'time' in user_response:
            respond(ctime())
        else:
            if(greeting(user_response)!=None):
                respond(greeting(user_response))
            else:
                print(user_response)
                respond(response(user_response)) #this is our Rawformattedpart
                sent_tokens.remove(user_response)
    else:
        flag=False
        respond("Bye! take care..")


['WE', 'ARE', 'IN', 'COMPUTERLAB', 'IN', 'THE', 'UNIVERSITY']
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
My name is chitti.I will answer your queries about Chatbots.        If you want to exit, say Bye!
Start talking
You said:stop
stop
I am sorry! I don't understand you
Start talking
I cannot hear you

I am sorry! I don't understand you
Start talking
I cannot hear you

I am sorry! I don't understand you


In [ ]:
import sklearn

In [ ]:
pip install scikit-learn